# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

[WARNING] graphlab.product_key: Unable to write current GraphLab Create license to /home/shan/.graphlab/config. Ensure that this user account                         has write permission to /home/shan/.graphlab/config to save the license for offline use.
[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1467193250.log


This non-commercial license of GraphLab Create is assigned to lightemit@gmail.com and will expire on January 20, 2017. For commercial licensing options, visit https://dato.com/buy/.


[WARNING] graphlab.deploy._session: Unable to create session in specified location: '/home/shan/.graphlab/artifacts'. Using: '/var/tmp/graphlab-shan/9958/tmp_session_592d40d2-4323-48cd-b31d-1f1224a88827'


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.299791     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.315623     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [12]:
nonzero = model_all.get("coefficients")["value"] != 0.0

In [10]:
model_all.get("coefficients").print_rows(19)
import numpy as np

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [15]:
print np.sum(nonzero.to_numpy())-1

5


In [16]:
print "bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above"

bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above


# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [17]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [18]:
l1 = np.logspace(1, 7, num = 13)

In [31]:
def fit_and_test(train, test, l1_penalty, feature_list):
    dic = []
    for penalty in l1_penalty:
        
        model = graphlab.linear_regression.create(train, target='price', features = feature_list,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty= penalty, verbose = False)
        
        output = test['price'].to_numpy() 
        predict = model.predict(test)
        predict = predict.to_numpy()
        err = predict - output
        RSS = np.dot(err,err)
        dic.append((penalty,RSS))
    
    return dic      
    

In [32]:
result = fit_and_test(training, validation, l1, all_features)


In [37]:
print min(result, key = lambda x:x[1])
result

(10.0, 625766285142460.5)


[(10.0, 625766285142460.5),
 (31.622776601683793, 625766285362393.88),
 (100.0, 625766286057885.25),
 (316.22776601683796, 625766288257224.88),
 (1000.0, 625766295212186.12),
 (3162.2776601683795, 625766317206080.62),
 (10000.0, 625766386760658.25),
 (31622.776601683792, 625766606749278.75),
 (100000.0, 625767302791635.0),
 (316227.76601683791, 625769507643885.88),
 (1000000.0, 625776517727024.5),
 (3162277.6601683795, 625799062845466.75),
 (10000000.0, 625883719085425.25)]

In [46]:
result2 = fit_and_test(training, testing, l1, all_features)
x = min(result2, key = lambda x:x[1])

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [48]:
print x[1]

1.56972779669e+14


In [47]:
print x[0]

10000000.0


In [38]:
print 10
print 625766285142460.5

10
6.25766285142e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [39]:
model = graphlab.linear_regression.create(training, target='price', features = all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty= 10.0, verbose = False)
        

In [41]:
model.get("coefficients").print_rows(18)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

In [64]:
model.["coefficients"].nnz()

SyntaxError: invalid syntax (<ipython-input-64-d386e157fe46>, line 1)

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [42]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [43]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [49]:
non_list = []
for p in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features = all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty= p, verbose = False)
    non_zero = model["coefficients"]["value"].nnz()
    non_list.append((p,non_zero))
    
        

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [50]:
non_list

[(100000000.0, 18),
 (127427498.57031322, 18),
 (162377673.91887242, 18),
 (206913808.11147901, 18),
 (263665089.87303555, 17),
 (335981828.62837881, 17),
 (428133239.8719396, 17),
 (545559478.11685145, 17),
 (695192796.17755914, 17),
 (885866790.41008317, 16),
 (1128837891.6846883, 15),
 (1438449888.2876658, 15),
 (1832980710.8324375, 13),
 (2335721469.0901213, 12),
 (2976351441.6313133, 10),
 (3792690190.7322536, 6),
 (4832930238.5717525, 5),
 (6158482110.6602545, 3),
 (7847599703.5146227, 1),
 (10000000000.0, 1)]

In [53]:
l1_penalty_min = 2976351441.6313133
l1_penalty_max = 3792690190.7322536

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [54]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [58]:
model_dic = {}
RSS_list = []
for p in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features = all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty= p, verbose = False)
    
    model_dic[p] = model
    nz = model["coefficients"]["value"].nnz()
    output = validation['price'].to_numpy()
    result = model.predict(validation)
    predict = result.to_numpy()
    err = predict - output
    RSS = np.dot(err, err)
    RSS_list.append((p,RSS,nz))

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [57]:
print RSS_list

[(2976351441.6313133, 966925692362085.25, 10), (3019316638.9524155, 974019450084555.0, 10), (3062281836.2735176, 981188367942454.12, 10), (3105247033.5946198, 989328342459473.75, 10), (3148212230.9157219, 998783211265890.0, 10), (3191177428.236824, 1008477167020093.8, 10), (3234142625.5579262, 1018298780553819.8, 10), (3277107822.8790283, 1028247992205977.2, 10), (3320073020.2001305, 1034616909232827.1, 8), (3363038217.5212326, 1038554735941040.4, 8), (3406003414.8423347, 1043237237871701.8, 8), (3448968612.1634369, 1046937488751710.8, 7), (3491933809.484539, 1051147625612862.4, 7), (3534899006.8056412, 1055992735342999.6, 7), (3577864204.1267428, 1060799531763287.6, 7), (3620829401.447845, 1065707689498230.1, 6), (3663794598.7689471, 1069464335425585.0, 6), (3706759796.0900493, 1073504549585599.2, 6), (3749724993.4111514, 1077632775581415.5, 6), (3792690190.7322536, 1081867592324112.5, 6)]


In [59]:
model1 = model_dic[2976351441.6313133]

In [60]:
temp_list = []
for item in RSS_list:
    if item[2] == 7:
        temp_list.append(item)
    else:
        pass

In [62]:
min(temp_list, key= lambda x:x[1])

(3448968612.1634369, 1046937488751710.8, 7)

In [63]:
model3 = model_dic[3448968612.1634369]
model3.get("coefficients").print_rows(18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |